# Gaia 3D Dust Extinction Map

from [Leike, Glatzle, Ensslin 2022](https://dx.doi.org/10.1051/0004-6361/202038169)

Download the file `mean_std.h5` from [here](http://cdsarc.u-strasbg.fr/viz-bin/cat/J/A+A/639/A138)

In [ ]:
from pathlib import Path
import imageio

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
import h5py

import  astro3d
from astro3d.image_stack import makeslice, process

plt.style.use([{'image.cmap':'gray_r'}])

Read the data

In [ ]:
data_path = Path('~/Desktop/Leike_Ensslin_2019/data/Leike_Glatzle_Ensslin_2020').expanduser()

with h5py.File(data_path / 'mean_std.h5') as f:
    data = f['mean'][()]
    print(f.attrs['information'].decode())

Set a normalization for the density

In [ ]:
vmax = data.max()
norm = LogNorm(1e-4 * vmax, vmax, clip=True)
#norm = Normalize(0.0003, 0.03 * vmax, clip=True)

Define function for the interpolation

In [ ]:
x = np.arange(data.shape[0])
y = np.arange(data.shape[1])
z = np.arange(data.shape[2])

In [ ]:
def f_interp(coords):
    return astro3d.fmodule.interpolate(x, y, z, data, coords)

Next, we need to look up the printer settings. Depending on the model, the (in-plane) x and y resolution is 600x300 DPI or 600x600 DPI. This distinction is crucial because printing a square image on 600x600 DPI would result also in a printed square, but on 600x300 DPI we would get a 1:2 aspect ratio.

In addition to that, we need to find out what layer thickness values the printer supports and pick one of them. Here we will proceed with a 600x300 DPI printer resolution and a layer thickness of 27 micron.

Furthermore, we want to print our data into a cubeoid of 4 cm height:

In [ ]:
height = 4 # this should be the total height of the printed cube in cm

# these are the values for the J850 Prime
dpi_x = 600
dpi_y = 300
dpi_z = 940 # 0.027 mm layer thickness = 2.54 / dpi_z

Define higher resolution grids on the printer resolution.

Apparently the image dimension should be even, so we add a single pixel if it isn't.

In [ ]:
n_z = int(height * dpi_z / 2.54)
n_x = int(n_z * len(x) / len(z) / dpi_z * dpi_x)
n_y = int(n_z * len(y) / len(z) / dpi_z * dpi_y)

n_x += n_x % 2 # add 1 to make it even if it isn't
n_y += n_y % 2 # add 1 to make it even if it isn't

# these are our new grids
x2 = np.linspace(0, data.shape[0] - 1, n_x)
y2 = np.linspace(0, data.shape[1] - 1, n_y)
z2 = np.linspace(0, data.shape[2] - 1, n_z)

coords = (x2, y2, z2)

print([*x2.shape, *y2.shape, *z2.shape])

### Iteration

We iterate over the entire 1850 layers and store the images in the path set by `output_dir`.

In [ ]:
output_dir = 'slices_gaia_2022'

In [ ]:
if type(norm).__name__ == 'Normalize':
    output_dir += '_linear'
elif type(norm).__name__ == 'LogNorm':
    output_dir += '_log'
else:
    output_dir += '_' + type(norm).__name__

Prepare output folder

In [ ]:
path = Path(astro3d.get_output()) / output_dir
path.mkdir(exist_ok=True)

First, select which layer index in the new z-grid to process for this example:

In [ ]:
iz = data.shape[-1] // 2
#iz = 0

This cell does the same as `makeslice`: interpolates one layer, creates and dithers the image and writes it to file

In [ ]:
makeslice(iz, z2, f_interp, coords, norm, path);

Let's check what this image looks like and compare to the data

In [ ]:
im = imageio.v2.imread(path / f'slice_{iz:04d}.png')

f, axs = plt.subplots(1, 2, figsize=(8, 4))

ax = axs[0]
ax.pcolormesh(x, y, data[:, :, z.searchsorted(z2[iz])].T, norm=norm, cmap='gray_r')
ax.set_xlabel('x [pixel]')
ax.set_ylabel('y [pixel]')
ax.set_aspect(1)
ax.set_title('original data')

ax = axs[1]
ax.imshow(im, vmin=0, vmax=255, cmap='gray')
ax.set_xlabel('x [pixel]')
ax.set_aspect(dpi_x / dpi_y)
ax.set_title('output image')

## Batch processing

all of the above can also be done in a loop with `process`:
normalizing with the given norm, up-scaling and saving to images. We'll just do this same one here by specifying the `iz` keyword.

In [ ]:
iz = np.arange(int(0.1 * dpi_z / 2.54)) # just the first millimeter

In [ ]:
process(data,
        height=height, dpi_x=dpi_x, dpi_y=dpi_y, dpi_z=dpi_z,
        output_dir=path,
        norm=norm,
        #iz=iz # comment this out to run the full stack
       )